<h1>A/B-testing<h1>

In [1]:
import pandas as pd
import sqlite3

<h2>create a connection to the database using the library sqlite3<h2>

In [70]:
conn = sqlite3.connect("../../data/checking-logs.sqlite")

<h2>using only one query for each of the groups, create two dataframes: test_results and control_results with the columns time and avg_diff and only two rows<h2>

In [66]:
query = '''
    WITH valid_uids AS (
        SELECT uid 
        FROM test
        GROUP BY uid
        HAVING MIN(unixepoch(first_commit_ts)) < MAX(unixepoch(first_view_ts))
            AND MAX(unixepoch(first_commit_ts)) > MIN(unixepoch(first_view_ts))
    )
    SELECT 'before',AVG((unixepoch(first_commit_ts)-deadlines.deadlines)/3600) AS avg_diff
    FROM test
    JOIN deadlines ON deadlines.labs = test.labname
    WHERE unixepoch(first_commit_ts) < unixepoch(first_view_ts) AND labname <> 'project1'
        AND uid IN (SELECT uid FROM valid_uids)
    UNION
    SELECT 'after',AVG((unixepoch(first_commit_ts)-deadlines.deadlines)/3600) AS avg_diff
    FROM test
    JOIN deadlines ON deadlines.labs = test.labname
    WHERE unixepoch(first_commit_ts) > unixepoch(first_view_ts) AND labname <> 'project1'
        AND uid IN (SELECT uid FROM valid_uids)
        
'''
test_results = pd.io.sql.read_sql(query, conn)

In [74]:
query = '''
    WITH valid_uids AS (
        SELECT uid 
        FROM control
        GROUP BY uid
        HAVING MIN(unixepoch(first_commit_ts)) < MAX(unixepoch(first_view_ts))
            AND MAX(unixepoch(first_commit_ts)) > MIN(unixepoch(first_view_ts))
    )
    SELECT 'before',AVG((unixepoch(first_commit_ts)-deadlines.deadlines)/3600) AS avg_diff
    FROM control
    JOIN deadlines ON deadlines.labs = control.labname
    WHERE unixepoch(first_commit_ts) < unixepoch(first_view_ts) AND labname <> 'project1'
        AND uid IN (SELECT * FROM valid_uids)
    UNION
    SELECT 'after',AVG((unixepoch(first_commit_ts)-deadlines.deadlines)/3600) AS avg_diff
    FROM control
    JOIN deadlines ON deadlines.labs = control.labname
    WHERE unixepoch(first_commit_ts) > unixepoch(first_view_ts) AND labname <> 'project1'
        AND uid IN (SELECT * FROM valid_uids)
        
'''
control_results = pd.io.sql.read_sql(query, conn)

In [67]:
control_results

,'before',avg_diff
0,after,-117.636364
1,before,-99.464286


In [76]:
test_results

,'before',avg_diff
0,after,-93.8500
1,before,-60.5625


<h2>close the connection<h2>

In [75]:
conn.close()

<h2>have the answer: did the hypothesis turn out to be true and the page does affect the students’ behavior?<h2>

We can not be sure, becouse control results are not the same